<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_4_2_%E1%84%86%E1%85%B5%E1%84%89%E1%85%A6%E1%84%8C%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%BC_Freezing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 감성 분류를 위한 BERT 미세조정


허깅페이스의 사전학습만 진행된 다국어 BERT 모델(bert-base-multilingual-cased)을 활용하여 감성분류를 위한 미세조정 테스크를 만들어 봅니다.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


## 데이터세트 구성

먼저 감정분류를 위한 데이터세트를 구성합니다. 미세조정도 사전학습과 동일하게 토큰화가 진행되어야 하고, 테스크에 맞게 입력과 라벨을 구성하여 만들어 줍니다.

예시에서는 [AIHUB](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=86) 한국어 감성 대화 말뭉치를 부분적으로 활용하여 6개의 감정을 분류하는 데이터세트를 구축합니다.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import sentencepiece as spm
import pandas as pd
import numpy as np

df = pd.read_csv(f'./train.csv')
sent_df = df[['label','HS01']].rename({'HS01':'text'},axis = 1).dropna()
sent_df

,label,text
0,E1,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,E1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,E1,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,E1,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,E1,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.
...,...,...
51623,E1,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.
51624,E3,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.
51625,E4,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.
51626,E3,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.


### 토크나이져 로드

허깅페이스를 활용하여 bert-base-multilingual-cased 모델을 로드할것 이므로 이에 맞는 토크나이져를 `AutoTokenizer` 객체를 통해 로드해 줍니다.

In [ ]:
from transformers import AutoTokenizer

model_name = "google-bert/bert-base-multilingual-cased"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer('점심 시간이 30분이 남았습니다')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

{'input_ids': [101, 9668, 71013, 9485, 96618, 10244, 37712, 10739, 8987, 119118, 119081, 48345, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

### Dataset 만들기

해당 예시에서는 감성분류를 위해 `토큰화된 텍스트 - 클래스 번호`형태로 매핑된 데이터를 만들어내는 Dataset을 구축합니다.

내부적으로 토큰화를 위해 미리 로드한 `AutoTokenizer`를 활용합니다.    
이때 학습을 위해 `max_length`(최대길이), `truncation`(자르기), `padding`(패딩)을 적용하여 토큰화 합니다.

허깅페이스의 BERT 모델은 기본적으로 입력 토큰(`input_ids`), 문장 토큰(`token_type_ids`), 어텐션 토큰(`attention_mask`) 3가지를 입력 받기 때문에 이에 맞게 Dataset의 출력을 딕셔너리로 구성해 주어야 합니다.   
해당 예시에서는 단일 문장만 사용하므로 문장 토큰을 생략하여 자동으로 0으로 할당되게 하고 추가로 라벨 또한 딕셔너리에 'labels' 키로 매핑해 출력해 줍니다.

In [ ]:
class SPDataSet(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.max_len = max_len
        self.df = df
        self.tokenizer = tokenizer
        self.class_name = {'E1':0, 'E6':1, 'E3':2, 'E5':3, 'E2':4, 'E4':5}

    def __getitem__(self, i):
        inp = str(self.df.iloc[i]['text'])
        tar = self.df.iloc[i]['label']
        # 라벨 인코딩
        tar = self.class_name[tar]

        # AutoTokenizer를 활용하여 입력 id, 패딩 마스크 생성
        encoding = self.tokenizer(
            inp,
            truncation=True,          # max_length 보다 길면 자르기
            padding="max_length",     # max_length 만큼 패딩
            max_length=self.max_len,  # max_length 설정
            return_tensors="pt"
        )

        item = {
            "input_ids": encoding["input_ids"].squeeze(),           # 입력 토큰
            "attention_mask": encoding["attention_mask"].squeeze(), # 어텐션 토큰
            "labels": torch.tensor(tar, dtype=torch.long)           # 분류 라벨
        }
        return item

    def __len__(self):
        return (len(self.df))


dataset = SPDataSet(sent_df, tokenizer, 60)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

for inp in dataloader:
    print(inp)
    break

{'input_ids': tensor([[   101,   9580, 118762,   9707,  96006,   8996,   9659,  40818,   8888,
          36553,  12453,   9647,  74311,   9095,  38688,  20173,  19653,   8888,
          36553,  14523, 119222,  12965,    119,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0],
        [   101,   9599,  24891,  32158,   9405,  61250,  27023,   9248,  16439,
          47869,   9490,  46572,  11903,    119,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,     

## 전체 파인튜닝을 통한 분류 학습

BERT 모델에 분류 미세조정을 할 때는 주로 모델 출력 중 문맥의 정보를 포함하는 [CLS] 토큰을 활용합니다.

[CLS] 토큰의 특성값으로부터 감정을 분류하는 분류기를 추가하고, 이에 맞춰 **BERT 인코더 전체와 분류기의 모든 가중치가 업데이트**되도록 전체 파인튜닝이 진행됩니다.

이 방식은 모델의 모든 레이어를 학습시키기 때문에 최고의 성능을 얻을 수 있지만, 많은 GPU 메모리와 학습 시간이 필요합니다.

### AutoModelForSequenceClassification 분류 모델

먼저 `AutoModelForSequenceClassification` 객체를 통해 내부적으로 미리 정의된 분류용 헤드(Classifier)가 자동으로 할당된 BERT 모델을 불러옵니다.

이는 인코더 출력인 [batch_size, seq_len, hidden_size]에서 [CLS] 위치(또는 pooler) 벡터를 뽑아 **Linear(num_labels)**를 통과시키는 단순 구조입니다.

이때, `num_labels` 인자에 몇개의 클래스가 있는지 전달하여 최종 레이어의 출력 형상을 지정합니다.

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# 분류기가 포함된 모델로 로드
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6, device_map="auto")
model

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

> 만약, 직접 구현한 FC Layer구조로 classifier을 구현하고 싶으면 AutoModelForSequenceClassification 에서 classifier 레이어를 직접 구현한 torch 레이어로 교체

    new_classifier = nn.Linear(in_features=model.classifier.in_features, out_features=5)
    model.classifier = new_classifier

### TrainingArguments 하이퍼파라미터 설정

허깅페이스의 프레임에 맞춰 학습을 진행할땐 `TrainingArguments` 객체를 통해 다양한 하이퍼파라미터 설정이 가능합니다.  

[TrainingArguments 인자](https://aiota.notion.site/TrainingArguments-1c7dc3286bf1800da7eac1ccfea46dfe?pvs=4)에서 어떤 파라미터 설정이 가능한지 확인할 수 있습니다. (업데이트에 따라 일부 달라질 수 있음)


In [ ]:
from transformers import TrainingArguments


# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir="./results",                 # 모델 저장 경로
    eval_strategy="steps",                  # 평가 전략 ("no", "steps", "epoch")
    eval_steps=200,                         # steps 평가 간격 (에포크인 경우는 1에폭마다 평가)
    num_train_epochs=5,                     # 에포크 수
    optim="adamw_torch",                    # 옵티마이져
    learning_rate=2e-5,                     # 학습률
    weight_decay=2e-5,                      # 가중치 감쇠
    per_device_train_batch_size=64,         # 학습 배치 크기
    per_device_eval_batch_size=64,          # 평가 배치 크기
    logging_steps=200,                      # 로그 출력 간격
    save_strategy = "steps",                # 모델 저장 전략 ("no", "steps", "epoch", "best")
    save_steps=1000,                        # 저장 간격
    save_total_limit=2,                     # 최대 저장수 (가장 좋은 모델만 남김)
    load_best_model_at_end=True,            # 학습 후 가장 평가가 좋은 모델 로드
    push_to_hub=False,                      # 모델을 허깅페이스 허브에 푸시 X
    report_to="none",                       # wandb 사용 X
)

dataset = SPDataSet(sent_df, tokenizer, 60)
generator1 = torch.Generator().manual_seed(42)
test_dataset, train_dataset = random_split(dataset, [0.2, 0.8], generator=generator1)
print(len(test_dataset), len(train_dataset))

10326 41302


### Trainer 모델 학습

허깅페이스는 학습 로직을 직접 구현할 필요없이 `TrainingArguments`로 만든 하이퍼파라미터를 적용한 학습을 수행하는 `Trainer` 객체를 제공합니다.

간단하게 `Trainer` 생성자에 모델과, TrainingArguments, 데이터세트를 입력하여 인스턴스를 만들고 `train()` 함수로 학습을 진행합니다.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# 학습 시작
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss,Validation Loss
200,1.544200,1.342909
400,1.310100,1.265899
600,1.260700,1.238095
800,1.196400,1.223677
1000,1.187300,1.207770
1200,1.168000,1.204767
1400,1.130800,1.209740
1600,1.109000,1.201867
1800,1.097600,1.211839
2000,1.084400,1.195726


TrainOutput(global_step=3230, training_loss=1.134262206015572, metrics={'train_runtime': 2911.9755, 'train_samples_per_second': 70.917, 'train_steps_per_second': 1.109, 'total_flos': 6367618998381600.0, 'train_loss': 1.134262206015572, 'epoch': 5.0})

## 프리징 하여 미세조정

모델의 모든 파라미터를 미세조정 하기에 너무 많은 리소스와 시간이 필요하다면, 모델의 특정 레이어를 학습하지 않게 동결하고 부분적인 레이어만 학습하게 설정하여 학습 가능합니다.



### 모델의 레이어 이름 확인

특정 레이어를 프리징 하기 위해선 먼저 레이어의 이름을 확인해주어야 합니다. 허깅페이스로 불러온 torch 모델에서 `named_parameters()` 함수를 통해 파라미터의 이름을 가져옵니다.

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model_frozen = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

# (파라미터 이름, 파라미터 텐서) 반환
for name, param in model_frozen.named_parameters():
    # 이름과 파라미터의 학습 가능 상태 확인
    print(f'layer : {name} / trainable : {param.requires_grad}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


layer : bert.embeddings.word_embeddings.weight / trainable : True
layer : bert.embeddings.position_embeddings.weight / trainable : True
layer : bert.embeddings.token_type_embeddings.weight / trainable : True
layer : bert.embeddings.LayerNorm.weight / trainable : True
layer : bert.embeddings.LayerNorm.bias / trainable : True
layer : bert.encoder.layer.0.attention.self.query.weight / trainable : True
layer : bert.encoder.layer.0.attention.self.query.bias / trainable : True
layer : bert.encoder.layer.0.attention.self.key.weight / trainable : True
layer : bert.encoder.layer.0.attention.self.key.bias / trainable : True
layer : bert.encoder.layer.0.attention.self.value.weight / trainable : True
layer : bert.encoder.layer.0.attention.self.value.bias / trainable : True
layer : bert.encoder.layer.0.attention.output.dense.weight / trainable : True
layer : bert.encoder.layer.0.attention.output.dense.bias / trainable : True
layer : bert.encoder.layer.0.attention.output.LayerNorm.weight / trainable

### 특정 레이어 프리징

파라미터를 프리징 즉, 학습 불가능 한 상태로 만들기 위해 파라미터의 `requires_grad` 변수를 False로 만들어줍니다.  

내가 원하는 레이어만을 프리징 하기 위해 전체 레이어의 `requires_grad`를 False로 만든 후, 반복문과 조건문을 통해 원하는 레이어 이름의 경우만 `requires_grad`를 True로 설정합니다. (반대 과정으로도 가능)



In [ ]:

# BERT 인코더 전체를 동결
for param in model_frozen.bert.parameters():
    param.requires_grad = False

# 특정 레이어(attention, pooler, classifier)만 학습
for name, param in model_frozen.named_parameters():
    if "attention" in name:
        param.requires_grad = True
    if "pooler" in name:
        param.requires_grad = True
    if "classifier" in name:
        param.requires_grad = True

for name, param in model_frozen.named_parameters():
    print(f'layer : {name} / trainable : {param.requires_grad}')




layer : bert.embeddings.word_embeddings.weight / trainable : False
layer : bert.embeddings.position_embeddings.weight / trainable : False
layer : bert.embeddings.token_type_embeddings.weight / trainable : False
layer : bert.embeddings.LayerNorm.weight / trainable : False
layer : bert.embeddings.LayerNorm.bias / trainable : False
layer : bert.encoder.layer.0.attention.self.query.weight / trainable : True
layer : bert.encoder.layer.0.attention.self.query.bias / trainable : True
layer : bert.encoder.layer.0.attention.self.key.weight / trainable : True
layer : bert.encoder.layer.0.attention.self.key.bias / trainable : True
layer : bert.encoder.layer.0.attention.self.value.weight / trainable : True
layer : bert.encoder.layer.0.attention.self.value.bias / trainable : True
layer : bert.encoder.layer.0.attention.output.dense.weight / trainable : True
layer : bert.encoder.layer.0.attention.output.dense.bias / trainable : True
layer : bert.encoder.layer.0.attention.output.LayerNorm.weight / trai

> 분류 레이어(classifier)는 반드시 학습 가능한 상태여야 합니다.

### 프리징된 모델 학습

학습은 동일하게 `Trainer`, `TrainingArguments`를 통해 학습 가능합니다.

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [ ]:
from transformers import Trainer, TrainingArguments
import evaluate

# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=200,
    num_train_epochs=20,
    optim="adamw_torch",
    learning_rate=2e-5,
    weight_decay=1e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    logging_steps=200,
    save_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"
)

dataset = SPDataSet(sent_df, tokenizer, 60)
generator1 = torch.Generator().manual_seed(42)
test_dataset, train_dataset = random_split(dataset, [0.2, 0.8], generator=generator1)

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model_frozen,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# 학습 시작
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
200,1.638900,1.473258,0.409258
400,1.427400,1.344937,0.471238
600,1.347000,1.301266,0.496029
800,1.281000,1.279143,0.509394
1000,1.266000,1.252788,0.518497
1200,1.238800,1.237674,0.523436
1400,1.218200,1.241885,0.527019
1600,1.195600,1.227372,0.529828
1800,1.175600,1.228275,0.526341
2000,1.178500,1.219777,0.535929


TrainOutput(global_step=3230, training_loss=1.2365729636083078, metrics={'train_runtime': 2187.0637, 'train_samples_per_second': 94.423, 'train_steps_per_second': 1.477, 'total_flos': 6367618998381600.0, 'train_loss': 1.2365729636083078, 'epoch': 5.0})

> 총 학습 시간이 BERT를 전체 미세조정한것에 비해 적게 걸린것을 확인할 수 있습니다.

# LLM 파인튜닝 방법 선택 기준 요약표

| 방법 | 언제 사용? (필요 상황) | 장점 | 단점 | GPU/메모리 요구 |
|------|---------------------------|--------|--------|------------------|
| **전체 미세조정 (Full Fine-tuning)** | - 데이터가 매우 많을 때 (수십~수백만)  <br> - 도메인 전체를 모델에 새로 학습시키고 싶을 때  <br> - 원본 모델을 완전히 다른 용도로 바꾸고 싶을 때 | - 성능 최고  <br> - 모델이 새로운 도메인에 완전히 최적화 | - 비용 가장 큼  <br> - GPU VRAM 많이 필요  <br> - 시간 오래 걸림 | 매우 높음 (A100 이상 권장) |
| **프리징된 미세조정 (Partial Freeze / Layer Freezing)** | - 데이터가 적을 때 (수천~수만)  <br> - 기존 모델의 능력 대부분 유지하고 싶을 때  <br> - 특정 기능만 약간 보완하고 싶을 때 | - 빠름  <br> - 파라미터 적어서 과적합 위험 ↓  <br> - GPU 부담 적음 | - 성능은 full fine-tuning보다 낮을 수 있음  | 중간 (T4 ~ L4도 가능) |
| **LoRA (Low-Rank Adaptation)** | - 중간 크기 모델(1B~70B) 튜닝 시  <br> - GPU 메모리가 많지 않을 때  <br> - 파라미터 효율적 미세조정 하고 싶을 때  <br> - 여러 버전의 어댑터를 저장하고 싶을 때 | - 파라미터 매우 절약  <br> - 속도 빠름  <br> - 성능도 꽤 좋음  <br> - 모델 원본 유지 가능 | - 특정 task에서는 Full FT보다 성능 낮을 수 있음 | 낮음~중간 (T4, L4, A10 가능) |
| **QLoRA (4bit Quantization + LoRA)** | - VRAM이 거의 없을 때 (8GB~16GB)  <br> - Colab/T4/로컬 환경에서 LLM 튜닝하고 싶을 때  <br> - 초저비용 튜닝이 목표일 때 | - GPU 메모리 요구 최저  <br> - 4bit로 줄이지만 성능 크게 안 떨어짐  <br> - 가장 저렴한 미세조정 방식 | - 4bit 변환 과정 추가  <br> - 극한의 정밀도가 필요한 task에서는 손실 발생 가능 | 매우 낮음 (T4, RTX 4060, Mac 등 가능) |




**요약**
- ✅ 데이터가 많고, 성능이 최우선 → Full Fine-tuning
- ✅ 성능·속도·안정성 골고루 → Partial Freeze Fine-tuning
- ✅ 적은 GPU로 고성능 확보 → LoRA
- ✅ 진짜 저렴하게 하고 싶다 → QLoRA